<a href="https://colab.research.google.com/github/Mohammed-majeed/Communicative-Robots/blob/main/AI2Thor_Robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import warnings,logging
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [3]:
!pip install --upgrade ai2thor --quiet
!pip install ai2thor-colab prior --upgrade &> /dev/null

In [4]:
import os
os.system('apt-get install xvfb')

import prior
dataset = prior.load_dataset("procthor-10k")

import ai2thor_colab
ai2thor_colab.start_xserver()

from ai2thor.controller import Controller
from PIL import Image

house = dataset["train"][11]
controller = Controller(scene=house, visibilityDistance=2, width=750, height=750)

img = Image.fromarray(controller.last_event.frame)

[AI2-THOR WARNING] There has been an update to ProcTHOR-10K that must be used with AI2-THOR version 5.0+. To use the new version of ProcTHOR-10K, please update AI2-THOR to version 5.0+ by running:
    pip install --upgrade ai2thor
Alternatively, to downgrade to the old version of ProcTHOR-10K, run:
   prior.load_dataset("procthor-10k", revision="ab3cacd0fc17754d4c080a3fd50b18395fae8647")


Loading test: 100%|██████████| 1000/1000 [00:01<00:00, 609.76it/s]


In [5]:
def Teleporting():
    event = controller.step(action="GetReachablePositions")
    reachable_positions = event.metadata["actionReturn"]
    event.metadata["actionReturn"]
    position = random.choice(reachable_positions)
    rotation = random.choice(range(360))
    event = controller.step(action="Teleport", position=position, rotation=rotation)
    img = Image.fromarray(controller.last_event.frame)
    return img

# Image to text

In [6]:
! pip install transformers -q

In [7]:
from transformers import pipeline

In [8]:
captioner = pipeline(model="ydshieh/vit-gpt2-coco-en")
caption = pipeline('image-to-text')
def image_to_caption(img):
    image_text= caption(img)[0]
    return image_text

In [9]:
!pip install nltk

In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def remove_stopwords_and_make_dataframe(caption):
    # Tokenize the generated text into words
    words = nltk.word_tokenize(caption)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Join the filtered words into a string
    filtered_text = ' '.join(filtered_words)

    # Create a Pandas DataFrame with the filtered caption
    df = pd.DataFrame({'text': [filtered_text]})
    return df

In [12]:
# Generate the caption and remove stop words
generated_caption = (image_to_caption(img))['generated_text']
filtered_caption = remove_stopwords_and_make_dataframe(generated_caption)
filtered_caption

,text
0,dining room table chairs table glass table


#text to classifier

In [13]:
!pip install simpletransformers

In [14]:
import random
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.special import softmax
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold

from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [23]:
# Load the dataset
url='https://github.com/Mohammed-majeed/Communicative-Robots/blob/main/dataset.csv'
df = pd.read_csv(url)
# Convert object names to lowercase
df['Object'] = df['Object'].str.lower()


ParserError: ignored

In [16]:
# inreach the dataset
labels = df['Label'].unique()
concatenated_dfs = []

for label in labels:
    label_objects = df[df['Label'] == label]
    temp = label_objects['Object'].tolist()
    random.shuffle(temp)
    label_objects['Object_1'] = temp
    random.shuffle(temp)
    label_objects['Object_2'] = temp
    concatenated_dfs.append(label_objects)

# Concatenate all DataFrames
result = pd.concat(concatenated_dfs, ignore_index=True)

temp = result['Object'].tolist()
random.shuffle(temp)
result['Object_3'] = temp

result.head()

,Object,Label,Object_1,Object_2,Object_3
0,air conditioner,bedroom,stool,air conditioner,wok
1,bed,bedroom,wallpaper,slippers,salt shaker
2,armchair,bedroom,picture,bookshelf,yogurt maker
3,bedsheet,bedroom,duvet,clothes valet,salad spinner
4,bedside table,bedroom,writing desk,stool,sponges


In [17]:
# Concatenate object columns into a single text column
result['text'] = result['Object'] + ' ' + result['Object_1'] + ' ' + result['Object_2'] + ' ' + result['Object_3']

# Encode the labels in the 'Label' column
encode_label={'bedroom':0, 'living room':1, 'bathroom':2, 'kitchen':3}
result['labels'] = result['Label'].map(encode_label)

df = result[['text', 'labels']]


# Shuffle the DataFrame
df_shuffled = df.sample(frac=1, random_state=42)  # Shuffling the DataFrame

# Define the split ratio (e.g., 80% for training, 20% for testing)
train_ratio = 0.8
test_ratio = 1 - train_ratio

# Calculate the number of samples for each split
train_size = int(train_ratio * len(df_shuffled))
test_size = len(df_shuffled) - train_size

# Split the DataFrame into training and test sets
train_df = df_shuffled.head(train_size)
test_df = df_shuffled.tail(test_size)

# Reset the index for the new DataFrames (optional)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [18]:
# calculate and print classification report
def print_classification_report(true_labels, predicted_labels, experiment_name):
    report = classification_report(true_labels, predicted_labels)
    print(f'Classification Report for {experiment_name}:')
    print(report)

def plot_confusion_matrix(true_labels, predictions, experiment_name):
    # Generate confusion matrix
    cm = confusion_matrix(true_labels, predictions)

    # Plot the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=result['Label'].unique(), yticklabels=result['Label'].unique())
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(f'Confusion Matrix for {experiment_name}')
    plt.show()

In [19]:
def fine_tune_and_evaluate(experiment_name, train_dataset, test_dataset,
                           model_type, model_case, retrain=False, num_folds=5):

    # Set up model directories and arguments
    best_model_directory = None
    model_directory = None
    if num_folds <= 1:
        # If num_folds is 1 or less, train and evaluate on the full training dataset without cross-validation
        model_directory = f"output/{model_type}/{model_case}/{experiment_name}"
        best_model_directory = model_directory + '/best_model'
    else:
        # Prepare k-fold cross-validation
        kf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
        model_directory = f"output/{model_type}/{model_case}/cv{num_folds}/{experiment_name}"
        best_model_directory = model_directory + '/best_model'

    model_arguments = ClassificationArgs(output_dir=model_directory,
                                         best_model_dir=best_model_directory,
                                         save_best_model=True,
                                         reprocess_input_data=True,
                                         overwrite_output_dir=True,
                                         no_cache=True,
                                         max_seq_length=32)

    # Initialize an empty DataFrame to store predictions from each fold
    val_df = pd.DataFrame()

    # Loop through the folds if using cross-validation
    if num_folds > 1:
        # Train or load the model based on the existence of the model directory
        if not os.path.exists(model_directory) or retrain:
            model = ClassificationModel(model_type, model_case, args=model_arguments, num_labels=4, use_cuda=False)
            for train_index, val_index in kf.split(train_dataset.text, train_dataset.labels):
                train_fold, val_fold = train_dataset.iloc[train_index], train_dataset.iloc[val_index]
                model.train_model(train_fold)

                # Make predictions on the validation dataset for this fold
                val_predictions, _ = model.predict(list(val_fold.text))

                val_fold[f"prediction{model_type}"] = val_predictions

                # Append predictions from val_fold to val_df
                val_df = pd.concat([val_df, val_fold], axis=0)

        else:
            model = ClassificationModel(model_type, model_directory, args=model_arguments, num_labels=4, use_cuda=False)

    else:
        # Train the model on the full training dataset without cross-validation
        # Train or load the model based on the existence of the model directory
        if not os.path.exists(model_directory) or retrain:
            model = ClassificationModel(model_type, model_case, args=model_arguments, num_labels=4, use_cuda=False)
            model.train_model(train_dataset)
        else:
            model = ClassificationModel(model_type, model_case, args=model_arguments, num_labels=4, use_cuda=False)

    # Make predictions on the test dataset
    test_predictions, test_probabilities = model.predict(list(test_dataset.text))

    # Calculate softmax probabilities for Soft Majority Voting
    softmax_probabilities = softmax(test_probabilities, axis=1)

    # Format the experiment name for display
    formatted_experiment_name = f'{model_type}" / "{model_case}" / "{experiment_name}'

    # Print and plot confusion matrix
    plot_confusion_matrix(test_dataset.labels, test_predictions, formatted_experiment_name)

    # Print classification report
    print_classification_report(test_dataset.labels, test_predictions, formatted_experiment_name)

    # Access the last layer of the BERT model and move it to the CPU
    last_layer_weights_cuda = model.model.classifier.weight.data
    last_layer_weights_cpu = last_layer_weights_cuda.cpu().numpy()

    # Return predictions, probabilities for ensembling, aggregated validation data, predictions, and indices as pandas DataFrames
    return test_predictions, softmax_probabilities, last_layer_weights_cpu, model


In [21]:
# Train and evaluate the model Bert in-domain
test_predictions, softmax_probabilities, last_layer_weights_cpu, model = fine_tune_and_evaluate(experiment_name='in-domain',
                                                                                                train_dataset = train_df, test_dataset = test_df,
                       model_type = 'bert', model_case='bert-base-uncased',retrain=True, num_folds=3)

  0%|          | 0/236 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Process ForkPoolWorker-3:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/usr/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


KeyboardInterrupt: ignored

# text to vector

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def encode_text_with_bert(text):
    # Load pre-trained BERT tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize the text
    input_ids = tokenizer.encode(text, add_special_tokens=True)

    # Convert input IDs to PyTorch tensor
    input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

    # Get BERT embeddings for the input text
    with torch.no_grad():
        outputs = model(input_ids)
        bert_embeddings = outputs.last_hidden_state.mean(dim=1)  # Use mean pooling to get a fixed-size embedding vector

    return bert_embeddings

In [ ]:
# text encoder
human_text = 'television bed table glass table'
img_text = (filtered_caption['text'])[0]

bert_embeddings = encode_text_with_bert(human_text)
bert_embeddings1 = encode_text_with_bert(img_text)

In [ ]:
# Calculate cosine similarity between the two text embeddings
similarity_score = cosine_similarity(bert_embeddings, bert_embeddings1)[0][0]
print("Cosine Similarity Score:", similarity_score)

# Test robot

In [ ]:
def main(human_text):

    human_text_room_prediction = ((model.predict([human_text]))[0]).tolist()
    robot_text_room_prediction = ((model.predict([(filtered_caption['text'])[0]]))[0]).tolist()
    print("Initial prediction",human_text_room_prediction,robot_text_room_prediction)
    itter = 50
    count=0
    while True:
        if robot_text_room_prediction[0] == human_text_room_prediction[0] or count==itter:
            break
        else:
            img_new = Teleporting()
            generated_caption = (image_to_caption(img_new))['generated_text']
            filtered_caption_new = remove_stopwords_and_make_dataframe(generated_caption)
            robot_text_room_prediction = ((model.predict([(filtered_caption_new['text'])[0]]))[0]).tolist()


        count+=1
        print(count,human_text_room_prediction,robot_text_room_prediction)

    if count == 0:
      print(human_text_room_prediction,robot_text_room_prediction)
      return img
    elif count <= itter:
      print(human_text_room_prediction,robot_text_room_prediction)
      return img_new
    else:
      print(human_text_room_prediction,robot_text_room_prediction)
      print('did not find the plase')

In [ ]:
human_text = 'bed bedsheet Bed'
human_text_room_prediction = ((model.predict([human_text]))[0]).tolist()
human_text_room_prediction
robot_text_room_prediction = ((model.predict([(filtered_caption['text'])[0]]))[0]).tolist()
robot_text_room_prediction

In [ ]:
from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE,SIG_DFL)
human_text = 'bed bedsheet Bed'

main(human_text)
